# Recessions

Note: Recessions cannot be determined algorithmically. Many factors need to be considered.

## Set-up

In [1]:
import pandas as pd
from pandas import DataFrame
import readabs as ra
from matplotlib.patches import Patch
from plotting import set_chart_dir, clear_chart_dir, finalise_plot

In [2]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# save charts in this notebook
CHART_DIR = "./CHARTS/Recessions/"
set_chart_dir(CHART_DIR)
clear_chart_dir(CHART_DIR)

# some plotting constants
CAT, CAT_GDP = "1364.0.15.003", "5206.0"
LFOOTER = "Australia. "
RFOOTER = f"ABS: {CAT}."
RFOOTER_GDP = f"ABS: {CAT_GDP}."
RFOOTER_BOTH = f"ABS: {CAT}, {CAT_GDP}."

# display charts in this notebook
SHOW = False

## Get the main data items

### Raw data acquisition

#### National Accounts

In [3]:
# construct population from components in National Accounts
# Note: in the national accounts the SA/CVM GDP/Capita only goes back to 1973.
# So we will construct population from the original series.

gdp = {
    "A2302460K": "GDP per capita: Chain volume measures (Orig)",
    "A2302459A": "Gross domestic product: Chain volume measures (Orig)",
}
gdp_data, _gdp_meta = ra.read_abs_series(
    "5206.0", gdp, single_excel_only="5206001_Key_Aggregates"
)
population = gdp_data["A2302459A"] / gdp_data["A2302460K"] * 1_000_000
population.name = "Population"

#### Modeller's database

In [4]:
wanted = {
    # All seasonally adjusted series
    "A2454517C": "Labour force",
    "A2454521V": "Unemployed",
    "A2454489F": "GDP",  # Chain volume measures, seasonally adjusted
}
model_d, model_m = ra.read_abs_series(CAT, wanted)

### Calculations

In [5]:
def build_dataset() -> DataFrame:
    """Build a dataset from the ABS data."""

    def two_negative_quarters(series):
        """Identify two consecutive negative quarters."""
        return (series < 0) & ((series.shift(-1) < 0) | (series.shift(1) < 0))

    data = model_d.rename(columns=wanted)
    data["Employed"] = data["Labour force"] - data["Unemployed"]
    data["Employment Growth"] = data["Employed"].pct_change(1) * 100
    data["Unemployment Rate"] = data["Unemployed"] / data["Labour force"] * 100
    data["GDP Growth"] = data["GDP"].pct_change() * 100
    data["GDP Recession"] = two_negative_quarters(data["GDP Growth"])
    data["Employment Recession"] = two_negative_quarters(data["Employment Growth"])
    data["GDP per Capita"] = data["GDP"] / population
    data["GDP per Capita Growth"] = data["GDP per Capita"].pct_change() * 100
    data["GDP per Capita Recession"] = two_negative_quarters(
        data["GDP per Capita Growth"]
    )

    # unemployment growth
    threshold = 1.0
    data["Rapid Unemployment Growth"] = data["Unemployment Rate"].diff(4) > threshold
    return data, threshold


DATA, THRESHHOLD = build_dataset()

## Plot

### Utility for plotting

In [6]:
def highlight(ax, series, color, alpha=0.5) -> None:
    """Add highlights to a chart based on a Boolean series."""

    shading, start, previous = False, None, None
    for index, item in series.items():
        if item and not shading:
            shading, start = True, index
        if shading and not item:
            ax.axvspan(start, previous, color=color, alpha=alpha)
            shading = False
        previous = index
    if shading:
        ax.axvspan(start, previous, color=color, alpha=alpha)

In [7]:
def hack_legend(ax, color, alpha, label) -> None:
    """Add a dummy entry to the legend to allow for a title."""

    patch = Patch(facecolor=color, alpha=alpha, label=label)
    handles, _labels = ax.get_legend_handles_labels()
    _lgd = ax.legend(handles=[handles[0], patch], loc="best", fontsize=8)

In [8]:
# constants
R_COLOUR, ALPHA = "darkorange", 0.5

### Technical recessions in GDP growth

In [9]:
def plot_gdp_tech_rec(data: DataFrame = DATA) -> None:
    """Plot GDP growth and technical recessions."""

    # Technical recession periods
    ax = data["GDP Growth"].plot(lw=0.75, c="blue")
    highlight(ax, data["GDP Recession"], color=R_COLOUR, alpha=ALPHA)
    hack_legend(ax, R_COLOUR, ALPHA, "Technical Recession")

    finalise_plot(
        ax,
        title="GDP Growth - Technical Recessions",
        ylabel="GDP Q/Q Growth (%)",
        y0=True,
        rfooter=RFOOTER,
        lfooter=f"{LFOOTER}Technical recession is two or more quarters of negative growth. "
        + "GDP is seasonally adjusted, chain volume measures. ",
        show=SHOW,
    )


plot_gdp_tech_rec()

### Technical recessions in employment growth

In [10]:
def plot_emp_tech_rec(data: DataFrame = DATA) -> None:
    """Plot employment growth and technical recessions."""

    ax = data["Employment Growth"].plot(lw=0.75, c="blue")
    highlight(ax, data["Employment Recession"], color=R_COLOUR, alpha=ALPHA)
    hack_legend(ax, R_COLOUR, ALPHA, "Technical Recession")

    finalise_plot(
        ax,
        title="Employment Growth - Technical Recessions",
        ylabel="Num. Employed Q/Q Growth (%)",
        y0=True,
        rfooter=RFOOTER,
        lfooter=f"{LFOOTER}Technical recession is two or more quarters of negative growth. ",
        show=SHOW,
    )


plot_emp_tech_rec()

### Technical recessions in GDP per capita growth

In [11]:
def plot_gdppc_tech_rec(data: DataFrame = DATA) -> None:
    """Plot GDP per capita growth and technical recessions."""

    ax = data["GDP per Capita Growth"].plot(lw=0.75, c="blue")
    highlight(ax, data["GDP per Capita Recession"], color=R_COLOUR, alpha=ALPHA)
    hack_legend(ax, R_COLOUR, ALPHA, "Technical Recession")

    finalise_plot(
        ax,
        title="GDP per Capita Growth - Technical Recessions",
        ylabel="GDP Q/Q Growth (%)",
        y0=True,
        rfooter=RFOOTER_BOTH,
        lfooter=f"{LFOOTER}Technical recession is two or more quarters of negative growth. "
        "GDP is seasonally adjusted, chain volume measures. ",
        show=SHOW,
    )


plot_gdppc_tech_rec()

### Periods of high growth in the unemployment rate

In [12]:
def plot_unemployment(data: DataFrame = DATA) -> None:
    """Plot the unemployment rate and highlight rapid increases."""

    # increase in the unemployment rate
    ax = data["Unemployment Rate"].plot(lw=0.75, c="blue")
    issues = data["Rapid Unemployment Growth"]
    highlight(ax, issues, color=R_COLOUR, alpha=ALPHA)
    hack_legend(
        ax, R_COLOUR, ALPHA, f"Growth > {THRESHHOLD} percentage point through the year"
    )

    finalise_plot(
        ax,
        title="Unemployment Rate - highlighting increases > 1.0pp in year",
        ylabel="Per cent of Labour Force",
        y0=True,
        rfooter=RFOOTER,
        lfooter=f"{LFOOTER} Seasonally adjusted. Quarterly data",
        show=SHOW,
    )


plot_unemployment()

## The end

In [13]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sun Jun 30 2024 19:15:35

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

readabs: 0.0.5
pandas : 2.2.2

Watermark: 2.4.3

